# Document Similarity in Natural Language Processing

## Overview
Document similarity is a fundamental task in NLP, allowing us to quantify how similar two pieces of text are. It has applications in search engines, document clustering, plagiarism detection, and recommendation systems. This notebook covers essential techniques for measuring document similarity, including:

 * Text Preprocessing
 * Cosine Similarity
 * Jaccard Similarity
 * Word Embeddings and Semantic Similarity

Let's dive in!

## Imports


In [1]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances
from sklearn.metrics import jaccard_score
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/oysterable/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Text Preprocessing

### Why Preprocess Text?

Text preprocessing is a critical step in NLP. Raw text data contains various elements that can introduce noise and reduce the effectiveness of similarity calculations. By preprocessing, we standardize and clean the text, ensuring that similarity measures capture meaningful information rather than irrelevant details.

Here are the key reasons why preprocessing is essential:
1. **Standardization**: Documents may contain the same information presented differently (e.g., "Machine Learning" vs. "machine learning"). Lowercasing ensures consistency.
2. **Noise Reduction**: Text can include punctuation, special characters, and numbers, which often don’t contribute meaningfully to understanding the text.
3. **Improved Tokenization**: Removing stopwords (e.g., "and", "is", "the") and stemming or lemmatizing words (reducing words to their root forms) makes it easier to match similar words.
4. **Reduced Vocabulary Size**: By removing irrelevant words and standardizing text, we reduce the vocabulary size, which improves memory efficiency and speeds up computations in NLP models.

Let’s walk through a simple preprocessing pipeline to clean our text.

In [2]:
# Our sample documents
documents = [
    "Raccoons are generally nocturnal animals.",
    "I love machine learning.",
    "Raccoons struggle to understand machine learning principles.",
    "Artificial intelligence & machine learning are fast-growing fields!"
]

import re
stop_words = set(stopwords.words('english'))

def preprocess(text):
    # Convert text to lowercase
    text = text.lower()

    # Remove punctuation and special characters
    text = re.sub(r'[^\w\s]', '', text)

    # Remove numbers
    text = re.sub(r'\d+', '', text)

    # Tokenize and remove stopwords
    words = text.split()
    words = [word for word in words if word not in stop_words]

    return ' '.join(words)

# Apply our preprocessing function to each document in the corpus
documents_clean = [preprocess(doc) for doc in documents]
documents_clean


['raccoons generally nocturnal animals',
 'love machine learning',
 'raccoons struggle understand machine learning principles',
 'artificial intelligence machine learning fastgrowing fields']

# Cosine Similarity

Cosine similarity is one of the most commonly used similarity metrics in text analysis and NLP. It measures the similarity between two vectors by calculating the cosine of the angle between them. This metric is particularly useful when dealing with high-dimensional data, such as text, where each document can be represented as a vector of word counts or TF-IDF scores.

## Why Cosine Similarity?

1. **Orientation Over Magnitude**: Cosine similarity focuses on the **angle** between two vectors, not their length. This makes it ideal for text data because document length can vary significantly (e.g., short vs. long articles). Cosine similarity allows us to compare documents based on their content and structure rather than just word counts.
   
2. **Normalized Similarity Score**: The cosine similarity score ranges from -1 to 1, where:
   - **1** indicates that the documents are identical in terms of content (same direction in vector space).
   - **0** indicates no similarity (orthogonal vectors).
   - **-1** (rare in NLP) would indicate completely opposite content (opposite directions).
   
   This normalized range makes cosine similarity easy to interpret, providing a consistent way to gauge similarity between documents.

3. **Efficient for Sparse Data**: Text data is often sparse, meaning each document contains only a small subset of possible words. Cosine similarity is efficient for high-dimensional, sparse vectors, as it only considers words present in the documents, which reduces computation time and memory usage.

4. **Suitability for Bag-of-Words Representations**: Cosine similarity works well with bag-of-words (BOW) or TF-IDF vector representations, which transform text into numerical vectors. In these vectorized forms, documents are represented by their word counts or weighted term frequencies, and cosine similarity effectively captures the similarity in terms of word usage and relevance.

## Formula

The cosine similarity between two vectors $A$ and $B$ is calculated as:

$
\text{cosine similarity} = \frac{A \cdot B}{||A|| \times ||B||}
$

where:
- $A \cdot B$ is the dot product of vectors $A$ and $B$.
- $||A||$ and $||B||$ are the magnitudes (or Euclidean norms) of vectors $A$ and $B$.

The dot product essentially measures the overlap in terms of word occurrences or term frequencies between the two documents, while the magnitudes normalize this overlap, allowing us to focus on the "direction" (content) rather than the "magnitude" (length) of each document.

### Practical Example

For example, consider two documents:
- **Document 1**: "Data science and machine learning are amazing."
- **Document 2**: "Machine learning and data science are exciting."

Both documents contain similar content, despite minor differences in word order and vocabulary. Cosine similarity will recognize this similarity by focusing on the common terms ("data science", "machine learning") and downplaying differences due to the exact wording or order, giving a high similarity score.


Now, let's compute the cosine similarity of our preprocessed documents.

In [3]:
# Vectorize the documents
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(documents_clean)

# Compute Cosine Similarity
cosine_sim_matrix = cosine_similarity(X)
cosine_sim_matrix

array([[1.        , 0.        , 0.20412415, 0.        ],
       [0.        , 1.        , 0.47140452, 0.47140452],
       [0.20412415, 0.47140452, 1.        , 0.33333333],
       [0.        , 0.47140452, 0.33333333, 1.        ]])

We can see that there is a positive similarity between documents 1 and 3, documents 2 and 3, and documents 3 and 4. Let's now try Jaccard similarity and compare results.

# Jaccard Similarity

Jaccard similarity is a metric used to measure the similarity between two sets. In NLP, it can be used to assess the similarity between two documents by treating each document as a set of unique words (or tokens). The Jaccard similarity score is defined as the ratio of the size of the intersection to the size of the union of the two sets. The score ranges from 0 to 1, where:

- **1** indicates that the documents have identical content (complete overlap in terms).
- **0** indicates no common terms between the documents.

## Why Use Jaccard Similarity?

Jaccard similarity is particularly useful for text analysis when we’re interested in understanding the overlap between unique terms in two documents. It’s a straightforward and interpretable metric that provides a sense of commonality, making it suitable for applications where exact word matches are important, such as:

1. **Plagiarism Detection**: Jaccard similarity can highlight shared terms between two documents, which can be an indicator of copied content.
2. **Duplicate Detection**: When identifying near-duplicate content, Jaccard similarity effectively captures the overlap of terms.
3. **Search and Information Retrieval**: By treating documents as sets of terms, Jaccard similarity can be used to find documents that share many of the same keywords as a given query.

## Jaccard Similarity Formula

The Jaccard similarity between two sets $A$ and $B$ is calculated as:

$$
\text{Jaccard Similarity} = \frac{|A \cap B|} {|A \cup B|}
$$

where:
- $|A \cap B|$ is the number of unique terms common to both documents (the intersection).
- $|A \cup B|$ is the total number of unique terms present in both documents combined (the union).

### Example Calculation

Consider two documents:
- **Document 1**: "machine learning is fascinating"
- **Document 2**: "machine learning is amazing"

If we treat each document as a set of unique words, we get:
- Document 1 set: {machine, learning, is, fascinating}
- Document 2 set: {machine, learning, is, amazing}

The intersection (common terms) is {machine, learning, is} and the union is {machine, learning, is, fascinating, amazing}. Therefore, the Jaccard similarity score is:

$$
\text{Jaccard Similarity} = \frac{3}{5} = 0.6
$$

This indicates moderate similarity based on shared terms.

## How Jaccard Similarity Compares to Cosine Similarity

While both Jaccard and cosine similarity are used to measure similarity between documents, they differ thusly:

1. **Focus on Unique Terms**: Jaccard similarity only considers the presence or absence of unique terms in each document, ignoring term frequency. This makes it sensitive to exact matches but less effective for capturing semantic similarity when terms overlap partially.

2. **Sensitivity to Document Length**: Jaccard similarity does not account for the frequency or importance of terms within a document. In contrast, cosine similarity can be used with term frequency or TF-IDF vectors, which account for both the importance and the frequency of terms, providing a more nuanced similarity score.

3. **Sparse vs. Dense Overlap**: Jaccard similarity is more suitable for binary, set-based comparisons and is often used in tasks where only exact term overlap matters. Cosine similarity, however, is generally preferred when documents contain a variety of words with some terms repeated. Cosine similarity captures the orientation of the entire vectorized representation of the document, making it more effective for capturing partial matches and overall content similarity.

### When to Use Each Metric

- **Use Jaccard Similarity** when looking for exact term matches or duplicate detection, where the goal is to measure pure overlap in vocabulary.
- **Use Cosine Similarity** when you want to capture broader semantic relationships and partial matches in term usage, such as in information retrieval, recommendation systems, and general document comparison.


In [4]:
# Compute the Jaccard Similarity between our cleaned documents
def jaccard_similarity(doc1, doc2):
    words_doc1 = set(doc1.split())
    words_doc2 = set(doc2.split())
    intersection = words_doc1.intersection(words_doc2)
    union = words_doc1.union(words_doc2)
    return len(intersection) / len(union)

# Output a similarity matrix
jaccard_sim_matrix = np.zeros((len(documents_clean), len(documents_clean)))
for i in range(len(documents_clean)):
    for j in range(len(documents_clean)):
        jaccard_sim_matrix[i, j] = jaccard_similarity(documents_clean[i], documents_clean[j])

jaccard_sim_matrix

array([[1.        , 0.        , 0.11111111, 0.        ],
       [0.        , 1.        , 0.28571429, 0.28571429],
       [0.11111111, 0.28571429, 1.        , 0.2       ],
       [0.        , 0.28571429, 0.2       , 1.        ]])

Lastly, let's talk about capturing the semantic meanings of words through word embeddings, the state of the art when it comes to document similarity.

# Word Embeddings and Semantic Similarity

Traditional similarity metrics like cosine similarity and Jaccard similarity rely on counting word overlap or using vectorized representations of words based on frequency (like TF-IDF). However, these methods lack the ability to capture the **semantic meaning** of words and phrases. **Word embeddings** are a powerful alternative that represent words in continuous vector spaces, capturing semantic relationships between words based on their usage across a large corpus.

## What Are Word Embeddings?

Word embeddings, such as **Word2Vec** and **GloVe**, are dense, low-dimensional vector representations of words that capture semantic meaning. Each word is mapped to a unique vector where semantically similar words are located close together in the vector space. For example, in a high-quality word embedding, words like "king" and "queen" or "doctor" and "nurse" would have vectors that are close to each other, even though they may not co-occur frequently.

With embeddings, we can represent an entire document as a single vector by taking the average (or another aggregation) of all word vectors in the document. This results in a **document embedding**, which we can then use to calculate similarity scores between documents in a way that captures both **syntactic** and **semantic** similarities.

## Calculating Semantic Similarity with Word Embeddings

1. **Convert each word to its embedding** using a pre-trained embedding model (e.g., GloVe or Word2Vec).
2. **Aggregate word embeddings** for each document by averaging the word vectors.
3. **Calculate similarity** between document embeddings using cosine similarity, which is effective for continuous, dense embeddings.

### Where do Word Embeddings Shine?

- **Captures Semantic Relationships**: Embeddings understand word meaning and context, enabling the model to recognize synonyms or related words even if they do not overlap directly in text.
- **Contextual Understanding**: Embeddings trained on large corpora learn context from neighboring words, making them effective for capturing language nuances and domain-specific terminology.
- **Improved Accuracy for Similar Documents**: Word embeddings generally outperform traditional similarity measures (cosine, Jaccard) in identifying conceptually similar documents, as they capture latent semantic relationships.

### Comparing Word Embeddings to Other Similarity Methods

| Similarity Measure       | Basis of Similarity         | Captures Semantic Meaning? | Sensitive to Term Frequency? | Typical Use Cases                                                                 |
|--------------------------|-----------------------------|----------------------------|------------------------------|----------------------------------------------------------------------------------|
| **Cosine Similarity**    | Angle between term vectors  | No                         | Yes                          | Document similarity with TF-IDF or bag-of-words, information retrieval           |
| **Jaccard Similarity**   | Overlap of unique terms     | No                         | No                           | Duplicate detection, exact matches, set-based similarity                         |
| **Euclidean Distance**   | Distance between term vectors | No                         | Yes                          | Similarity for numerical, dense vectors, sometimes used in clustering            |
| **Word Embeddings**      | Dense vector representation of words | Yes                        | No                           | Semantic similarity, recommendation systems, contextual document similarity      |

### Key Differences Between Word Embeddings and Traditional Similarity Metrics

1. **Semantic Awareness**:
   - **Traditional Metrics**: Cosine and Jaccard similarity focus on **exact term overlap** or **relative frequency** but lack semantic understanding. For instance, "doctor" and "physician" may be considered dissimilar due to lack of term overlap.
   - **Word Embeddings**: Word embeddings capture **semantic relationships** between words. With embeddings, "doctor" and "physician" would have similar vectors, leading to higher similarity scores even if these words do not overlap directly.

2. **Sensitivity to Document Size**:
   - **Traditional Metrics**: Jaccard similarity measures pure overlap without considering document length, while cosine similarity considers relative term frequency but does not normalize for semantics.
   - **Word Embeddings**: By averaging word vectors, embeddings offer a **document-size invariant representation** that remains consistent regardless of document length.

3. **Handling Synonyms and Context**:
   - **Traditional Metrics**: They do not handle synonyms or related terms well since these metrics depend solely on word overlap and frequency.
   - **Word Embeddings**: Embeddings trained on large corpora capture context, making them robust to synonymy and semantic nuances.



In [5]:
import gensim.downloader as api

# Load pre-trained word vectors (e.g., GloVe)
word_vectors = api.load("glove-wiki-gigaword-50")

def document_vector(doc):
    """Create document vectors by averaging word vectors. Ignore words not in vocabulary."""
    words = doc.split()
    word_vecs = [word_vectors[word] for word in words if word in word_vectors]
    return np.mean(word_vecs, axis=0) if word_vecs else np.zeros(50)

# Generate document embeddings
doc_vectors = np.array([document_vector(doc) for doc in documents_clean])

# Compute cosine similarity on document embeddings
embedding_cosine_sim_matrix = cosine_similarity(doc_vectors)
embedding_cosine_sim_matrix

[==================================================] 100.0% 66.0/66.0MB downloaded


array([[1.0000001 , 0.35509798, 0.49974078, 0.41246545],
       [0.35509798, 1.0000001 , 0.8658411 , 0.8233106 ],
       [0.49974078, 0.8658411 , 1.        , 0.8203855 ],
       [0.41246545, 0.8233106 , 0.8203855 , 1.        ]], dtype=float32)

We can see that using word embeddings provides a much less sparse similarity matrix, compared to using similarity metrics on non-vectorized documents.

In [9]:
print(X[0]) #precense or absence of a word

  (0, 11)	1
  (0, 4)	1
  (0, 9)	1
  (0, 0)	1


In [7]:
doc_vectors

array([[ 0.7660975 , -0.44717848, -0.6827333 , -0.797     ,  0.38407725,
         0.5674025 , -0.4964825 , -1.0409074 ,  0.4642875 ,  0.04673575,
         0.29193574,  0.24002124,  1.2548425 ,  0.2546455 ,  0.26352   ,
         0.334715  ,  0.515765  , -0.01965251, -0.9713912 , -0.33076373,
        -0.6501725 , -0.25954026,  0.678575  ,  0.34542498,  0.27669   ,
        -0.00653751, -0.0926375 ,  0.2078175 ,  0.59477496, -0.31192   ,
         1.4462376 ,  0.48617497,  0.461235  , -0.7615825 , -0.0473525 ,
         0.336995  ,  0.00744382, -0.353587  , -0.68226504,  0.2047475 ,
        -0.80321   , -0.118825  ,  0.3130725 ,  1.1131275 ,  0.7319038 ,
        -0.12825425,  0.02745751, -0.45865   ,  0.09084225,  0.07816499],
       [-0.09196667,  0.27134   ,  0.01536667, -0.16772334,  0.312633  ,
         0.46026668, -0.16913335, -0.35260653, -0.16155367,  0.51632994,
         0.02772134,  0.18204999, -0.31387898, -0.15556599, -0.03769   ,
        -0.174287  , -0.21033466,  0.7958934 , -0.

In [ ]:
# Define the two vectors
print(embedding_cosine_sim_matrix[0])

v1=np.array(embedding_cosine_sim_matrix[1])
v2=np.array(embedding_cosine_sim_matrix[2])

# References

1. https://courses.cs.washington.edu/courses/cse573/12sp/lectures/17-ir.pdf
2. https://www.newscatcherapi.com/blog/ultimate-guide-to-text-similarity-with-python
3. https://www.polarsparc.com/xhtml/Document-Similarity-NLP.html

In [6]:
# A simple example of a lexicon (dictionary of sentiment scores)
lexicon = {
    "great": 2,
    "excellent": 3,
    "bad": -2,
    "terrible": -3,
    "not": -1  # negation handling
}

# Text preprocessing (removing punctuation, lowercasing, tokenizing)
def preprocess(text):
    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text)
    return text.split()

# Calculating sentiment score
def calculate_sentiment(text, lexicon):
    tokens = preprocess(text)
    sentiment_score = 0
    negation_flag = False

    for token in tokens:
        if token in lexicon:
            score = lexicon[token]
            # Handle negation
            if negation_flag:
                score = -score
                negation_flag = False  # Reset after applying negation
            sentiment_score += score
        if token == "not":  # Set flag for negation
            negation_flag = True

    # Determine sentiment category
    if sentiment_score > 0:
        sentiment = "Positive"
    elif sentiment_score < 0:
        sentiment = "Negative"
    else:
        sentiment = "Neutral"

    return sentiment, sentiment_score

# Example usage
text = "The service was not bad, but the food was excellent."
sentiment, score = calculate_sentiment(text, lexicon)
print(f"Text: '{text}'")
print(f"Sentiment: {sentiment} (Score: {score})")


Text: 'The service was not bad, but the food was excellent.'
Sentiment: Positive (Score: 4)
